In [3]:
require 'torch'
require 'nn'
require 'rnn'

In [4]:
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

In [5]:
-- Simulating the data
qValues = torch.rand(10, 2)

-- Generating the max values and getting the indices
qMax, qindx = torch.max(qValues, 2)

--- I want to select the qindx elements for each row
actions = torch.zeros(10, 2):scatter(2, qindx, torch.ones(qValues:size()))

-- Simulating streams and queries
sentences = genNbyK(10, 15, 1, 100)
queries = genNbyK(10, 4, 1, 100)

In [6]:
sentences:size()

 10
 15
[torch.LongStorage of size 2]



In [16]:
indx = torch.range(1, 10)

In [31]:
actions:select(2, 1)

 1
 0
 0
 0
 0
 1
 0
 1
 1
 1
[torch.DoubleTensor of size 10]



In [35]:
indx, indx:index(1, torch.LongTensor({1, 3}))

  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
[torch.DoubleTensor of size 10]

 1
 3
[torch.DoubleTensor of size 2]



In [40]:
actions:select(2, 1):resize(10, 1):byte()

 1
 0
 0
 1
 0
 1
 0
 1
 0
 1
[torch.ByteTensor of size 10x1]



In [39]:
ms = nn.MaskedSelect()
mask = torch.ByteTensor({{1, 0}, {0, 1}})
input = torch.DoubleTensor({{10, 20}, {30, 40}})
print(input)
print(mask)

out = ms:forward({input, mask})
print(out)


 10  20
 30  40
[torch.DoubleTensor of size 2x2]

 1  0
 0  1
[torch.ByteTensor of size 2x2]

 10
 40
[torch.DoubleTensor of size 2]



In [52]:
indx:resize(10, 1), actions:select(2,1):resize(10, 1):byte()

  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
[torch.DoubleTensor of size 10x1]

 1
 0
 0
 1
 0
 1
 0
 1
 0
 1
[torch.ByteTensor of size 10x1]



In [58]:
sumindx = nn.MaskedSelect():forward({indx, actions:select(2,1):resize(10, 1):byte()})  --- sentences

In [65]:
sumindx

  1
  4
  6
  8
 10
[torch.DoubleTensor of size 5]



In [68]:
sumindx = torch.LongTensor(5):copy(sumindx)

# We've selected the relevant summaries very quickly

In [70]:
sentences:index(1, sumindx)

   2   91   58   97   11   54   95   35   62   43   13   74   13   12   54
   3   89   70   30   32    8   35   82   54   81   56   92   41   75   46
 100    4   63   73   63   88   76   62   70   11   81   88   44   86   27
  99   40   59   20   70    5    2   78   64   94   60   35   98    6   77
   3   69    5   66   38   57   21   62   91   48   87   14   25   71   14
[torch.LongTensor of size 5x15]



In [12]:
function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

In [13]:
model = buildModel('bow', 100, 50, 'f1', false, false)

Running bag-of-words model to learn f1	


In [14]:
model:forward({streams, queries, })

...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:67: 
In 1 module of nn.Sequential:
In 1 module of nn.ParallelTable:
In 1 module of nn.Sequential:
.../torch/install/share/lua/5.1/rnn/LookupTableMaskZero.lua:12: attempt to index local 'input' (a nil value)
stack traceback:
	.../torch/install/share/lua/5.1/rnn/LookupTableMaskZero.lua:12: in function <.../torch/install/share/lua/5.1/rnn/LookupTableMaskZero.lua:7>
	[C]: in function 'xpcall'
	...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...avierarceo/torch/install/share/lua/5.1/nn/Sequential.lua:44: in function <...avierarceo/torch/install/share/lua/5.1/nn/Sequential.lua:41>
	[C]: in function 'xpcall'
	...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...erarceo/torch/install/share/lua/5.1/nn/ParallelTable.lua:12: in function <...erarceo/torch/install/share/lua/5.1/nn/ParallelTable.lua:10>
	[C]: in function 'xpcall'
	...javierarceo/torch/in